In [1]:
GPU = True

if GPU:
    import cupy as np
    np.cuda.set_allocator(np.cuda.MemoryPool().malloc)
else:
    import numpy as np

# for time series split
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
# optimizer
class Adam:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = [], []
            for param in params:
                self.m.append(np.zeros_like(param))
                self.v.append(np.zeros_like(param))
        
        self.iter += 1
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)

        for i in range(len(params)):
            self.m[i] += (1 - self.beta1) * (grads[i] - self.m[i])
            self.v[i] += (1 - self.beta2) * (grads[i]**2 - self.v[i])
            
            params[i] -= lr_t * self.m[i] / (np.sqrt(self.v[i]) + 1e-7)

In [3]:
def MSE(y, t):
    return 0.5*np.mean((y-t)**2)

class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x<=0)
        y = x.copy()
        y[self.mask] = 0

        return y
    
    def backward(self, dy):
        dy[self.mask] = 0
        dx = dy

        return dx


class TimeFC:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        N, D = x.shape

        reshaped_x = x.reshape(N, -1)
        y = np.dot(reshaped_x, W) + b
        
        self.x = x
        y = y.reshape(N, -1)
        return y

    def backward(self, dy):
        W, b = self.params
        x = self.x
        N, D = x.shape

        dy = dy.reshape(N, -1)
        reshaped_x = x.reshape(N, -1)

        db = np.sum(dy, axis=0)
        dx = np.matmul(dy, W.T)
        dW = np.matmul(reshaped_x.T, dy)
        
        dx = dx.reshape(*x.shape)

        self.grads[0][...] = dW
        self.grads[1][...] = db

        return dx

class TimeMSE:
    def __init__(self):
        self.params, self.grads = [], []
        self.activation = ReLU()
        self.cache = None

    def forward(self, xs, ts):
        N, V = xs.shape
        xs = xs.reshape(N, V)
        xs = self.activation.forward(xs)
        ts = ts.reshape(N, V)

        loss = MSE(xs, ts)
        self.cache = (ts, xs, (N, V))

        return loss

    def backward(self, dy = 1):

        ts, xs, (N,  V) = self.cache
        
        dx = dy * (xs - ts) / (N)

        dx = self.activation.backward(dx)
        dx = dx.reshape(N , V)
        return dx

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [5]:
class LSTM:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev, c_prev):
        # Affine transformation (Wx[f, g, i, o], Wh[f, g, i, o], b[f, g, i, o])
        Wx, Wh, b = self.params
        N, H = h_prev.shape

        A = np.matmul(x, Wx) + np.matmul(h_prev, Wh) + b

        # slice for gates and get
        forget = A[:, :H]       # NxH
        get = A[:, H:2*H]
        input = A[:, 2*H:3*H]
        output = A[:, 3*H:4*H]

        forget = sigmoid(forget)   # forget gate
        get = np.tanh(get)        # new memory
        input = sigmoid(input)    # input gate
        output = sigmoid(output)    # output gate

        c_next = (c_prev * forget) + (get * input)
        h_next = np.tanh(c_next) * output

        self.cache = (x, h_prev, c_prev, input, forget, get, output, c_next)
        return h_next, c_next

    def backward(self, dh_next, dc_next):
        Wx, Wh, b = self.params
        x, h_prev, c_prev, input, forget, get, output, c_next = self.cache

        # chain rule
        do = dh_next * np.tanh(c_next)
        do_s = do * output*(1-output)
        dt = dh_next * output
        dt_c = dt * (1-(np.tanh(c_next)**2))

        di = dt_c * get
        dg = dt_c * input
        di_s = di * input*(1-input)
        dg_t = dg * (1-(get**2))

        dc_prev = dt_c * forget
        df = dt_c * c_prev
        df_s = df * forget*(1-forget)

        dA = np.hstack((df_s, dg_t, di_s, do_s))

        db = np.sum(dA, axis = 0)
        dWh = np.matmul(h_prev.T, dA)
        dh_prev = np.matmul(dA, Wh.T)
        dWx = np.matmul(x.T, dA)
        dx = np.matmul(dA, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev, dc_prev

In [6]:
# Time LSTM
class TimeLSTM:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None      # for LSTM layer
        self.h, self.c = None, None
        self.dh = None
        self.stateful = stateful

    def set_state(self, h, c=None):
        self.h, self.c = h, c

    def reset_state(self):
        self.h, self.c = None, None

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape      # mini-batch, time length, Dimension
        self.T = T
        H = Wh.shape[0]         # Wh (H, 4H) H: hidden size
        
        self.layers = []        # for stacking LSTM layer (horizontal)
        hs = np.empty((N, T, H), dtype='f')   # for save (h0 ... ht)
        
        # if not stateful, initialize h and c
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
        if not self.stateful or self.c is None:
            self.c = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = LSTM(*self.params)
            self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c)
            
            hs[:, t, :] = self.h
            self.layers.append(layer)
        # many to one
        hs = hs[:, -1, :] # return last states
        return hs
        
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, H = dhs.shape
        T = self.T
        D = Wx.shape[0]

        dxs = np.empty((N, T, D), dtype='f')
        dh, dc = 0, 0
        dh = dhs

        grads = [0, 0, 0]   #dWx, dWh, db
        for t in reversed(range(self.T)):  # BPTT
            layer = self.layers[t]
            # dx, dh, dc = layer.backward(dhs[:,t ,:] + dh, dc)
            dx, dh, dc = layer.backward(dh, dc)
            dxs[:, t, :] = dx
            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        
        self.dh = dh
        return dxs
    

In [7]:
class Model:
    def __init__(self, time_size, hidden_size, feature_size):
        T, H, F = time_size, hidden_size, feature_size
        H2 = 64
        rand = np.random.randn

        # weights (Xavier)
        lstm_Wx = (rand(F, 4*H)/ np.sqrt(F)).astype('f')
        lstm_Wh = (rand(H, 4*H)/ np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4*H).astype('f')

        # He
        fc_W1 = (rand(H, H2)/ np.sqrt(H/2)).astype('f')
        fc_b1 = np.zeros(H2).astype('f')
        fc_W2 = (rand(H2, 1)/ np.sqrt(H2/2)).astype('f')
        fc_b2 = np.zeros(1).astype('f')


        # layer
        self.layers = [
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True),
            TimeFC(fc_W1, fc_b1),
            TimeFC(fc_W2, fc_b2)
        ]
        self.loss_layer = TimeMSE()

        self.params, self.grads = [], []

        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads


    def predict(self, xs):
        xs = np.array(xs)
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs

    def forward(self, xs, ts):
        xs = np.array(xs)
        ts = np.array(ts)
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dy = 1):
        dy = self.loss_layer.backward(dy)
        for layer in reversed(self.layers):
            dy = layer.backward(dy)
        return dy


    def fit(self, train_X=None, train_y=None, learning_rate=0.01, epochs=10, batch_size=32, verbose=0):
        optimizer = Adam(learning_rate)

        data_size = train_X.shape[0]
        max_iters = data_size//batch_size

        for epoch in range(1, epochs+1):
            idx = numpy.random.permutation(numpy.arange(data_size))
            data_X = train_X[idx]
            data_y = train_y[idx]

            epoch_loss = 0
            start_time=time.time()
            
            for iter in range(max_iters):
                batch_x = data_X[iter*batch_size:(iter+1)*batch_size]
                batch_y = data_y[iter*batch_size:(iter+1)*batch_size]

                loss = self.forward(batch_x, batch_y)
                self.backward()
                params, grads = self.params, self.grads
                optimizer.update(params, grads)

                epoch_loss += loss
            avg_loss = epoch_loss/max_iters

            if verbose:
                duration = start_time-time.time()
                print(f'epoch:{epoch}/{epochs}, 시간:{duration:.2f}[s], loss:{avg_loss:.5f}')

    def reset_state(self):
        self.layers[0].reset_state()

In [22]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [10]
    batch_size = [64]
    learning_rate = [0.1, 0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config, verbose=0):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = Model(time_size=24, hidden_size=100, feature_size=8)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate, verbose=verbose)
    return model

def MAE_metric(x, t):
    return np.mean(np.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [9]:
import pandas as pd
from datetime import datetime

df_parser = lambda x: datetime.strptime(x, '%Y %m %d %H')    # string to datetime
# data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv'
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv'
df = pd.read_csv(data_url, sep=',', parse_dates=[['year', 'month', 'day', 'hour']], date_parser=df_parser, index_col=0)

del df['No']
df.columns = ['pm2.5', 'dewp', 'temp', 'pres', 'cbwd','wind_speed', 'snow', 'rain']
df = df[24:]            # NaN values in first 24hours
df.head()

,pm2.5,dewp,temp,pres,cbwd,wind_speed,snow,rain
year_month_day_hour,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [24]:
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy
import time

dataset = df.values
encoder = LabelEncoder()        # for cbwd : wind direction
dataset[:, 4] = encoder.fit_transform(dataset[:, 4])
dataset = dataset.astype('float')

last_hours = 24
features = 8
del_idx = last_hours*features + 1
del_cols = [i for i in range(del_idx, del_idx+features-1)]
reframed = series_to_img(dataset, last_hours)
reframed.drop(reframed.columns[del_cols], axis=1, inplace=True)
reframed.reset_index(drop=True, inplace=True)
print(reframed.head())

n_splits = 3
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=last_hours).split(reframed)

next(train_test_split)

configs = model_config()
history = []
i = 1

print('config : epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1
    print(train_cv_indices)
    # split x, y data
    train_cv_X, train_cv_y = reframed.iloc[train_cv_indices, :-1].values, reframed.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = reframed.iloc[test_cv_indices, :-1].values, reframed.iloc[test_cv_indices, -1].values

    test_length = len(test_cv_X)

    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, last_hours, features)
    val_X = val_X.reshape(-1, last_hours, features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outter loop
    train_cv_X = train_cv_X.reshape(-1, last_hours, features)
    test_cv_X = test_cv_X.reshape(-1, last_hours, features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    model_fit(train_cv_X, train_cv_y, configs[0], verbose=0)
    break

    print(train_cv_X.shape, test_cv_X.shape, train_cv_y.shape, test_cv_y.shape)


   var1(t-24)  var2(t-24)  var3(t-24)  ...  var7(t-1)  var8(t-1)  var1(t)
0       129.0       -16.0        -4.0  ...        3.0        0.0     90.0
1       148.0       -15.0        -4.0  ...        4.0        0.0     63.0
2       159.0       -11.0        -5.0  ...        5.0        0.0     65.0
3       181.0        -7.0        -5.0  ...        6.0        0.0     55.0
4       138.0        -7.0        -5.0  ...        7.0        0.0     65.0

[5 rows x 193 columns]
config : epochs, batch_size, learning_rate
fold : 1/3
[    0     1     2 ... 15012 15013 15014]


KeyboardInterrupt: ignored

In [ ]:
configs = model_config()
model_fit(train_X, train_y, configs[0])

In [ ]:
import time

# 80 train, 20 test
batch_size = 64
hidden_size = 100
time_size = 24

learning_rate = 0.001
epochs = 2

model = Model(time_size, hidden_size, 8)
optimizer = Adam(learning_rate)

data_size = train_X.shape[0]
max_iters = data_size//batch_size

# traning 

loss_arr=[]

for epoch in range(1, epochs+1):
    idx = numpy.random.permutation(numpy.arange(data_size))
    train_X = train_X[idx]
    train_y = train_y[idx]
    losses = 0

    start_time = time.time()
    for iter in range(max_iters):
        batch_x = train_X[iter*batch_size:(iter+1)*batch_size]
        batch_y = train_y[iter*batch_size:(iter+1)*batch_size]

        loss = model.forward(batch_x, batch_y)
        model.backward()
        params, grads = model.params, model.grads
        optimizer.update(params, grads)

        losses += loss.item()

    avg_loss = losses/max_iters
    
    if epoch%1==0:    
        during = time.time() - start_time
        print(f'epoch : {epoch}/{epochs}, 시간 {during:.2f}[s],  loss: {avg_loss:.5f}')
    loss_arr.append(avg_loss)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(epochs), loss_arr, label='train-Loss')
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
## last
import numpy

model.reset_state()
true = test_y
predict = model.predict(test_X)
if GPU:
    predict = np.asnumpy(predict)

def graph(x_range, iters):
    for i in range(iters):
        plt.title('PM 2.5')
        plt.plot(range(x_range), true[i*x_range:(i+1)*x_range], label='True')
        plt.plot(range(x_range), predict[i*x_range:(i+1)*x_range], label='Predict')
        plt.xlabel('times')
        plt.ylabel('pm 2.5')
        plt.legend(loc='upper right')
        plt.show()

graph(150, 3)


In [ ]:
import numpy
# mse, rmse, mae
def MAE_metric(x, t):
    return numpy.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)
print('*evaluate*')
print('mse : ', MSE_metric(predict, true))
print('rmse : ', np.sqrt(MSE_metric(predict, true)))
print('mae : ', MAE_metric(predict, true))